<a href="https://colab.research.google.com/github/ecenn/C23-PS260/blob/main/heart_model_fix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Setup**

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

#**Data**

In [5]:
file = '/content/heart-dataset.csv'
df = pd.read_csv(file, ',')

<ipython-input-5-01e52f0c332e>:2: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df = pd.read_csv(file, ',')


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253680 entries, 0 to 253679
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   HeartDiseaseorAttack  253680 non-null  float64
 1   HighBP                253680 non-null  float64
 2   HighChol              253680 non-null  float64
 3   CholCheck             253680 non-null  float64
 4   BMI                   253680 non-null  float64
 5   Smoker                253680 non-null  float64
 6   Stroke                253680 non-null  float64
 7   Diabetes              253680 non-null  float64
 8   PhysActivity          253680 non-null  float64
 9   Fruits                253680 non-null  float64
 10  Veggies               253680 non-null  float64
 11  HvyAlcoholConsump     253680 non-null  float64
 12  AnyHealthcare         253680 non-null  float64
 13  NoDocbcCost           253680 non-null  float64
 14  GenHlth               253680 non-null  float64
 15  

In [7]:
df = df.reindex(columns=['HeartDiseaseorAttack', 'Sex', 'Age', 'BMI', 'HighBP', 'HighChol', 'Stroke', 'Diabetes', 'Smoker', 'HvyAlcoholConsump', 'PhysActivity', 'Fruits', 'Veggies', 'DiffWalk'])

In [8]:
names = {'HeartDiseaseorAttack':'sakitJantung', 'Sex':'kelamin', 'Age':'umur', 'BMI':'bmi', 'HighBP':'tekananDarah', 'HighChol':'kolesterol', 'Stroke':'stroke', 'Diabetes':'diabetes', 'Smoker':'rokok', 'HvyAlcoholConsump':'alkohol', 'PhysActivity':'olahraga', 'Fruits':'buah', 'Veggies':'sayur', 'DiffWalk':'susahJalan'}
df = df.rename(columns=names)

In [9]:
cols = df.columns

for names in cols:
  print(names)
  unique_values = df[names].unique()
  print(unique_values)

sakitJantung
[0. 1.]
kelamin
[0. 1.]
umur
[ 9.  7. 11. 10.  8. 13.  4.  6.  2. 12.  5.  1.  3.]
bmi
[40. 25. 28. 27. 24. 30. 34. 26. 33. 21. 23. 22. 38. 32. 37. 31. 29. 20.
 35. 45. 39. 19. 47. 18. 36. 43. 55. 49. 42. 17. 16. 41. 44. 50. 59. 48.
 52. 46. 54. 57. 53. 14. 15. 51. 58. 63. 61. 56. 74. 62. 64. 66. 73. 85.
 60. 67. 65. 70. 82. 79. 92. 68. 72. 88. 96. 13. 81. 71. 75. 12. 77. 69.
 76. 87. 89. 84. 95. 98. 91. 86. 83. 80. 90. 78.]
tekananDarah
[1. 0.]
kolesterol
[1. 0.]
stroke
[0. 1.]
diabetes
[0. 2. 1.]
rokok
[1. 0.]
alkohol
[0. 1.]
olahraga
[0. 1.]
buah
[0. 1.]
sayur
[1. 0.]
susahJalan
[1. 0.]


In [10]:
# replace umur start from 0
df['umur'] = df['umur'].replace([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0],
 [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0]).astype(float)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253680 entries, 0 to 253679
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   sakitJantung  253680 non-null  float64
 1   kelamin       253680 non-null  float64
 2   umur          253680 non-null  float64
 3   bmi           253680 non-null  float64
 4   tekananDarah  253680 non-null  float64
 5   kolesterol    253680 non-null  float64
 6   stroke        253680 non-null  float64
 7   diabetes      253680 non-null  float64
 8   rokok         253680 non-null  float64
 9   alkohol       253680 non-null  float64
 10  olahraga      253680 non-null  float64
 11  buah          253680 non-null  float64
 12  sayur         253680 non-null  float64
 13  susahJalan    253680 non-null  float64
dtypes: float64(14)
memory usage: 27.1 MB


In [ ]:
# (TARGET)
# sakitJantung
# [0. 1.] = Tidak, Ya

# (FEATURES)
# kelamin
# [0. 1.] = Perempuan, Lakilaki
# umur
# [0. sampai 12.] = '18-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59','60-64', '65-69', '70-74', '75-79', '80 or older'
# bmi
# [float]
# tekananDarah
# [0. 1.] = Normal, Tinggi
# kolesterol
# [0. 1.] = Normal, Tinggi
# stroke
# [0. 1.] = Tidak, Ya
# diabetes
# [0. 1. 2.] = Tidak, Prediabetes(Hampir Diabetes) , Ya
# rokok
# [0. 1.] = Tidak, Ya
# alkohol
# [0. 1.] = Tidak, Ya
# olahraga
# [0. 1.] = Tidak, Ya
# buah
# [0. 1.] = Tidak, Ya
# sayur
# [0. 1.] = Tidak, Ya
# susahJalan
# [0. 1.] = Tidak, Ya

#**Model**

In [12]:
X = df.drop('sakitJantung', axis=1)
Y = df['sakitJantung']

x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [13]:
a = X.shape
b = Y.shape
c = x_train.shape
d = y_train.shape
e = x_test.shape
f = y_test.shape
print(f'X: {a} | x_train: {c} | x_test: {e}')
print(f'Y: {b} | y_train: {d} | y_test: {f}')

X: (253680, 13) | x_train: (202944, 13) | x_test: (50736, 13)
Y: (253680,) | y_train: (202944,) | y_test: (50736,)


In [14]:
input_shape = (x_train.shape[1],)
model = Sequential([Dense(128, activation='relu', input_shape=input_shape),
                    Dense(1, activation='sigmoid')])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
5074/5074 [==============================] - 14s 3ms/step - loss: 0.2924 - accuracy: 0.9059 - val_loss: 0.2596 - val_accuracy: 0.9065
Epoch 2/10
5074/5074 [==============================] - 14s 3ms/step - loss: 0.2529 - accuracy: 0.9068 - val_loss: 0.2524 - val_accuracy: 0.9063
Epoch 3/10
5074/5074 [==============================] - 13s 3ms/step - loss: 0.2487 - accuracy: 0.9071 - val_loss: 0.2505 - val_accuracy: 0.9067
Epoch 4/10
5074/5074 [==============================] - 12s 2ms/step - loss: 0.2472 - accuracy: 0.9070 - val_loss: 0.2494 - val_accuracy: 0.9066
Epoch 5/10
5074/5074 [==============================] - 11s 2ms/step - loss: 0.2464 - accuracy: 0.9070 - val_loss: 0.2489 - val_accuracy: 0.9065
Epoch 6/10
5074/5074 [==============================] - 12s 2ms/step - loss: 0.2460 - accuracy: 0.9070 - val_loss: 0.2487 - val_accuracy: 0.9060
Epoch 7/10
5074/5074 [==============================] - 13s 3ms/step - loss: 0.2456 - accuracy: 0.9071 - val_loss: 0.2484 - val_ac

#**Performance Evaluation**

In [15]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

1586/1586 [==============================] - 3s 2ms/step - loss: 0.2451 - accuracy: 0.9078
Test Loss: 0.24506935477256775, Test Accuracy: 0.9077578186988831


In [16]:
# input format >>
# kelamin[0,1]
# umur[0,1,2,3,4,5,6,7,8,9,10,11,12]
# bmi[float]
# tekananDarah[0,1]
# kolesterol[0,1]
# stroke[0,1]
# diabetes[0,1,2]
# rokok[0,1]
# alkohol[0,1]
# olahraga[0,1]
# buah[0,1]
# sayur[0,1]
# susahJalan[0,1]

# Healthy
print("The result should be between 0 and 0.5 ( Healthy / Low Risk Heart Disease ):")
model.predict([[1.0, 1.0, 22.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0]])

# Not Healthy
# print("The result should be between 0.5 and 1 ( Unhealthy / High Risk Heart Disease ):")
# model.predict([[1.0, 5.0, 85.2, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0]])

The result should be between 0 and 0.5 ( Healthy / Low Risk Heart Disease):
1/1 [==============================] - 0s 139ms/step


array([[1.4102562e-18]], dtype=float32)

#**Save Model**

In [18]:
import os
from tensorflow.python.saved_model.save import save

save_dir = os.path.join('/tmp/', 'newfile/saved_model.h5')
model.save(save_dir)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
!pip install tensorflowjs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 83.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 89.5 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.14.0
    Uninstalling tensorflow-estimator-2.14.0:
      Successfully uninstalled tensorflow-estimator-2.14.0
  Attempting uninstall: keras
    Found existing installation: keras 2.14.0
    Uninstalling keras-2.14.0:
      Successfully uninstalled keras-2.14.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.14.1
    Uninstalling tensorboard-2.14.1:
      Successfully uninstalle

In [22]:
!cd /tmp/newfile/
!tensorflowjs_converter --input_format=keras /tmp/newfile/saved_model.h5 /tmp/heart_converted

2023-12-10 16:18:30.580369: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-10 16:18:30.580429: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-10 16:18:30.581845: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-10 16:18:31.914883: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [23]:
!zip -r /tmp/heart_model.zip /tmp/heart_converted

  adding: tmp/heart_converted/ (stored 0%)
  adding: tmp/heart_converted/model.json (deflated 70%)
  adding: tmp/heart_converted/group1-shard1of1.bin (deflated 7%)
